In [1]:
import pickle

# Charger les données prétraitées à partir du fichier pickle
with open('donnees_traitees.pkl', 'rb') as f:
    df = pickle.load(f)


In [2]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer, OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier

# Supprimer les lignes contenant des valeurs manquantes
df_clean = df
# Séparer les variables indépendantes et la variable dépendante
X = df_clean.drop(['MIS_Status'], axis=1)
y = df_clean['MIS_Status']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Créer le transformateur pour les colonnes catégorielles
categorical_features = ['NAICS', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Créer le transformateur pour les colonnes numériques
numerical_features =['ApprovalFY', 'Term', 'GrAppv', 'NewExist']
numerical_transformer = SimpleImputer(strategy='mean')



# Créer le ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
        ])

# Créer le pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', XGBClassifier())])

param_grid = {
    'classifier__n_estimators': [ 300, 500, 1000],  # Nombre d'arbres dans le modèle
    'classifier__max_depth': [5, 10, 15],            # Profondeur maximale de chaque arbre
    'classifier__learning_rate': [0.01,0.1] # Taux d'apprentissage
}

# Créer le GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Exécuter la recherche sur la grille
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres trouvés:")
print(grid_search.best_params_)

# Utiliser le meilleur modèle trouvé
best_model = grid_search.best_estimator_

# Prédire les classes pour les données de test avec le meilleur modèle
y_pred = best_model.predict(X_test)

# Calculer les métriques avec le meilleur modèle
print("\nMatrice de confusion:")
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

print("\nRapport de classification:")
print(classification_report(y_test, y_pred))

print("\nAccuracy:")
print(accuracy_score(y_test, y_pred))

# Afficher la matrice de confusion
cf_matrixD = ConfusionMatrixDisplay(cf_matrix)
cf_matrixD.plot()

ValueError: y contains previously unseen labels: '262'